# Saving and Reloading Simulations

Here we show how to save a binary field with all the parameters for particles and the simulation's REBOUNDx effects. We begin with a one planet system subject to general relativity corrections:

In [1]:
import rebound
import numpy as np
sim = rebound.Simulation()
sim.add(m=1., hash="star") # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630, hash="planet") # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame
ps = sim.particles
print("t = {0}, pomega = {1}".format(sim.t, sim.particles[1].pomega))

t = 0.0, pomega = 0.0


We add GR, and after integrating, see that the pericenter has moved:

In [2]:
import reboundx
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr")
rebx.add_force(gr)
from reboundx import constants
gr.params["c"] = constants.C
sim.integrate(10.)
print("t = {0}, pomega = {1}".format(sim.t, sim.particles[1].pomega))

t = 10.0, pomega = 3.418184768435803e-06


Now we add some arbitrary parameters to the planet:

In [3]:
rebx.register_param('a', 'REBX_TYPE_INT')
rebx.register_param('b', 'REBX_TYPE_INT')

ps[1].params["a"] = 1
ps[1].params["b"] = 2

To save the simulation, we have to save a REBOUND binary to save the simulation, and a REBOUNDx binary to save the parameters and REBOUNDx effects:

In [4]:
sim.save("reb.bin")
rebx.save("rebx.bin")

We can now reload the simulation and the rebx instance:

In [5]:
sim2 = rebound.Simulation("reb.bin")
rebx2 = reboundx.Extras(sim2, "rebx.bin")

/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:466: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


In [6]:
ps2 = sim2.particles
gr2 = rebx2.get_force("gr")
print("Original: {0}, Loaded: {1}".format(ps[1].params["a"], ps2[1].params["a"]))
print("Original: {0}, Loaded: {1}".format(ps[1].params["b"], ps2[1].params["b"]))
print("Original: {0}, Loaded: {1}".format(gr.params["c"], gr2.params["c"]))

Original: 1, Loaded: 1
Original: 2, Loaded: 2
Original: 10065.32012038219, Loaded: 10065.32012038219


We can now continue integrating as usual, and see that the pericenter continues to precess, so the GR effect has been successfully added to the loaded simulation:

In [7]:
sim2.integrate(20.)
print("t = {0}, pomega = {1}".format(sim2.t, sim2.particles[1].pomega))

t = 20.0, pomega = 6.317055505806844e-06
